In [1]:
import pandas as pd

In [2]:
from datetime import datetime

In [4]:
file_path = 'origin_data'
df = pd.read_csv(file_path, parse_dates=['event_time'])

In [5]:
# 2. 기간 필터링 예시 (특정 월의 데이터만 사용한다고 가정)
start_date = '2024-09-01'
end_date = '2024-09-30'
df = df[(df['event_time'] >= start_date) & (df['event_time'] <= end_date)]

In [8]:
# event_time을 datetime 형식으로 변환
df['event_time'] = pd.to_datetime(df['event_time'])

# 날짜 단위로 변환
df['date'] = df['event_time'].dt.date

# 이벤트 집계
event_count = df.groupby(['date', 'item_id', 'item_name'])['inter'].count().reset_index()
event_count.rename(columns={'inter': 'event_count'}, inplace=True)

In [9]:
# 4. 이동 평균으로 급증 분석
event_count['7_day_avg'] = event_count.groupby('item_id')['event_count'].transform(lambda x: x.rolling(window=7, min_periods=1).mean())
event_count['increase_rate'] = event_count['event_count'] / event_count['7_day_avg']

In [10]:
# 급증한 상품 찾기: 3배 이상 급증한 경우 필터링
sudden_increase_items = event_count[event_count['increase_rate'] >= 4]

In [11]:
# 결과 확인
print(sudden_increase_items[['date', 'item_name', 'event_count', '7_day_avg', 'increase_rate']])

              date              item_name  event_count   7_day_avg  \
45402   2024-09-03      디아지오)기네스흑맥주캔440ML         1843  433.500000   
47787   2024-09-03        오비)스텔라캔500ML(신)         2850  671.666667   
61200   2024-09-04  신선특별시)친환경 청상추(120g/봉)          355   77.166667   
61216   2024-09-04  신선특별시)친환경 꽃상추(120g/봉)          356   67.857143   
61346   2024-09-04       신선특별시)햇배(3~5입/봉)          251   43.166667   
...            ...                    ...          ...         ...   
579557  2024-09-28       편스)찬또트리플치즈미트&파스타          126   25.857143   
584222  2024-09-28          편스토랑)찬또배기된장술밥          759  187.857143   
584253  2024-09-28     편스토랑)장민호의장어추어탕600G          114   22.714286   
589689  2024-09-29              푸나무 소비뇽블랑         1256  288.857143   
593786  2024-09-29   글렌알라키10CS 배치11 700ml          103   19.000000   

        increase_rate  
45402        4.251442  
47787        4.243176  
61200        4.600432  
61216        5.246316  
61346        5.814672  
...            